In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

url = "https://raw.githubusercontent.com/AykutErenSahin/Estimating-Daily-User-Count/main/Model%20test%20verisi.csv"

holidays_2023 = [
    "2023-01-01", "2023-04-21", "2023-04-22", "2023-04-23", "2023-04-24",
    "2023-05-01", "2023-05-19", "2023-06-28", "2023-06-29", "2023-06-30",
    "2023-07-01", "2023-07-02", "2023-07-15", "2023-08-30", "2023-10-29",
    "2024-01-01", "2024-04-10", "2024-04-11", "2024-04-12", "2024-04-13",
    "2024-04-23", "2024-05-01", "2024-05-19", "2024-06-16", "2024-06-17",
    "2024-06-18", "2024-06-19", "2024-06-20", "2024-07-15", "2024-08-30",
    "2024-10-29"
]

date_range = pd.date_range(start="01-01-2023", end="31-12-2024", freq="D")

data = {
    "Yil": date_range.year,
    "YilinGunu": date_range.dayofyear,
    "Ay": date_range.month,
    "AyinGunu": date_range.day,
    "AyinIlkGunu": date_range.is_month_start,
    "AyinOrtasi": date_range.is_month_start,                                                #Asıl kolon bunu üzerine yazılacak yer tutmak için burada
    "AyinSonGunu": date_range.is_month_end,
    "Hafta": date_range.isocalendar().week,
    "HaftaninGunu": date_range.dayofweek,
    "HaftaSonu": date_range.dayofweek >= 5,
    "TatilMi": [str(date.date()) in holidays_2023 for date in date_range],
    "TatilOncesi": [date - timedelta(days=1) in holidays_2023 for date in date_range],
    "TatilSonrasi": [date + timedelta(days=1) in holidays_2023 for date in date_range],
    "Trend" : date_range.dayofweek,                                                         #Asıl kolon bunu üzerine yazılacak yer tutmak için burada
    "Tarih": date_range
}
List = np.arange(731)

df = pd.DataFrame(data, index=List)

def get_week_number(date):
    return (date.dayofyear - 1) // 7 + 1

df['Hafta'] = df['Tarih'].apply(get_week_number)

def trend(month):
    return month*3
        
df['Trend'] = df['Ay'].apply(trend)

def is_mid_month(day):
    return day == 15

df['AyinOrtasi'] = df['AyinGunu'].apply(is_mid_month)

dff = pd.read_csv(url, sep=';')
result = pd.concat([df, dff], axis=1)

prophet_df = result[['Tarih', 'Gerceklesen']].dropna()
prophet_df = prophet_df.rename(columns={'Tarih': 'ds', 'Gerceklesen': 'y'})
prophet_df['ds'] = pd.to_datetime(prophet_df['ds'])
prophet_df['y'] = prophet_df['y'].astype(float)

# Prophet modelini oluştur ve eğit
model = Prophet()
model.fit(prophet_df)

future = model.make_future_dataframe(periods=175)               # Eksik kalan 175 gün için tahmin yaptıracağız
forecast = model.predict(future)

future['Tahmin'] = forecast['yhat'].round().astype(int)         # Tahminleri future dataframe'ine ekle

plt.figure(figsize=(12, 6))
plt.plot(prophet_df['ds'], prophet_df['y'], label='Gerçek Veriler')
plt.plot(future['ds'], future['Tahmin'], label='Tahmin', color='red')
plt.title('Günlük Kullanici Tahmini')
plt.xlabel('Tarih')
plt.ylabel('Kullanici Sayisi')
plt.legend()
plt.show()


model.plot_components(forecast)                             # Bileşen grafikleri
plt.show()

prophet_df.plot(x='ds', y='y')                              # Orijinal veriler
plt.title('Orijinal Veriler')
plt.show()

final_result = pd.merge(result, future[['ds', 'Tahmin']], left_on='Tarih', right_on='ds', how='left')
final_result = final_result.drop('ds', axis=1)

display(final_result)